In [8]:
import os
import pandas as pd
from pathlib import Path

# plotly
import plotly.graph_objects as go
import plotly.express as px

# scoot functions
from cleanair.scoot import (
    generate_fp
    load_model_from_file
    load_processed_data_from_file
    plotly_results
)

In [9]:
name = "test"

# make the figure directory
xpfp = "./experiments"  # root to filepaths directory
figure_dir = os.path.join(xpfp, name, "figures")
Path(figure_dir).mkdir(exist_ok=True)

# load the raw csv
normal_df = pd.read_csv(generate_fp(name))
lockdown_df = pd.read_csv(generate_fp(name, prefix="lockdown"))

In [10]:
detector_id = "N00/002e1"
prefix = "normal"           # normal or lockdown

# load data for this prefix and detector
model = load_model_from_file(name, detector_id, prefix=prefix)
X, Y = load_processed_data_from_file(name, detector_id, prefix=prefix)

In [11]:
color_counts = 'C0'
label_counts = 'N'
color_estimated_counts = 'red'
label_estimated_counts = '$\hat{N}$'

In [13]:
index = 0
sensor_df = normal_df.loc[normal_df.detector_id == detector_id]

fig = go.Figure()

fig.add_trace(go.Scatter(x=sensor_df['measurement_start_utc'], y=sensor_df['n_vehicles_in_interval'],
                    mode='lines+markers',
                    name='lines+markers')
)

fig.update_layout(title='Timeseries of sensor {id}'.format(id=detector_id),
                xaxis_title="Datetime",
                yaxis_title="# of vechicles per hour",
                font=dict(
                    size=16)
)

fig.show()

In [6]:
# THIS IS BUGGY
## Computes percentage cover (see Virginia's pdf for details)
def percentage_coverage(model,test_inputs,Ytest,quantile:int = 0.95, num_samples:int = 10,num_pertubations: int = 100):
    # Number of times total counts were within 90th percentile
    coverage_events = 0
    
    # Loop over pertubations
    for i in range(num_pertubations):

        # Change seed
        np.random.seed(i)
        
        # Sample from latent function (intensity)
        intensity_sample = np.exp(model.predict_f_samples(test_inputs,num_samples))
        # Compute emprical distribution of counts
        empirical_count_distribution = np.random.poisson(intensity_sample)
        
        # Total number of actual counts
        total_counts = np.sum(Ytest)
       
        # Compute upper and lower quantiles from the empirical distribution of counts
        upper_q = np.quantile(np.sum(samples[:,:,0],axis=1),quantile)
        lower_q = np.quantile(np.sum(samples[:,:,0],axis=1),1-quantile)
    
        # Add 1 - if total counts are within quantile, 0 - otherwise
        coverage_events += int((total_counts < upper_q) & (total_counts > lower_q))
        binary = int((total_counts < upper_q) & (total_counts > lower_q)) # this is kept for debugging (remove afterwards)

    return empirical_count_distribution, binary, total_counts, upper_q, lower_q # this is kept for debugging (remove afterwards)
    return coverage_events/num_pertubations # this should be the output after debugging

In [15]:
plotly_results(sensor_df, detector_id, model, X, num_samples=1000)